<a href="https://colab.research.google.com/github/yuezhang23/nlp/blob/main/shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interpreting Classifier Weights

In this experiment, you will train models to distringuish examples of two different genres of Shakespeare's plays: comedies and tragedies. (We'll ignore the histories, sonnets, etc.) Since he died four hundred years ago, Shakespeare has not written any more plays—although scraps of various other works have come to light. We are not, therefore, interested in building models simply to help categorize an unbounded stream of future documents, as we might be in other applications of text classification; rather, we are interested in what a classifier might have to tell us about what we mean by the terms “comedy” and “tragedy”.

You will start by copying and running your `createBasicFeatures` function from the experiment with movie reviews. Do the features the classifier focuses on tell you much about comedy and tragedy in general?

You will then implement another featurization function `createInterestingFeatures`, which will focus on only those features you think are informative for distinguishing between comedy and tragedy. Accuracy on leave-one-out cross-validation may go up, but it more important to look at the features given the highest weight by the classifier. Interpretability in machine learning, of course, may be harder to define than accuracy—although accuracy at some tasks is hard enoough.

In [88]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut
import numpy as np

In [89]:
#read in the shakespeare corpus
def readShakespeare():
  raw = requests.get("https://raw.githubusercontent.com/dasmiq/cs6120-assignment2/refs/heads/main/shakespeare_plays.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  #remove histories from the data, as we're only working with tragedies and comedies
  corpus = [entry for entry in corpus if entry["genre"] != "history"]
  return corpus

This is where you will implement two functions to featurize the data:

In [90]:
# TODO: Implement createBasicFeatures
# NB: The current contents are for testing only
# This function should return:
#  -a sparse numpy matrix of document features
#  -a list of the correct genre for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix.

# This function should create a feature representation using all tokens that
# contain an alphabetic character.
import json
import requests
import re
from scipy.sparse import lil_matrix, csr_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut
import numpy as np
def createBasicFeatures(corpus):
  #Your code here
  texts = [text['text'] for text in corpus]
  genres = [text['genre'] for text in corpus]

  tokens = []
  vocab = set()
  for txt in texts:
    tks = re.split(r'\W+', txt)
    tks = [tk.lower() for tk in tks if tk and tk.isalpha()]
    tokens.append(tks)
    # collect unique tokens
    vocab.update(set(tks))

  vocab = list(vocab)
  vocab_idx = {v: i for i, v in enumerate(vocab)}
  # create a sparse matrix
  texts = lil_matrix((len(corpus), len(vocab)), dtype=np.float64)
  for i in range(len(tokens)):
    tks = tokens[i]
    for tk in tks:
      texts[i, vocab_idx[tk]] += 1
  texts = texts.tocsr()
  return texts,genres,vocab

In [98]:
# TODO: Implement createInterestingFeatures. Describe your features and what
# they might tell you about the difference between comedy and tragedy.
# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
def createInterestingFeatures(corpus):
  #Your code here
  genres = [text['genre'] for text in corpus]

  def countFeaturesByNgram(text, n):
    """
      Extract n-grams from text and build a vocabulary of unique n-grams.

      Args:
          text (str): Input text to process
          n (int): Size of n-grams to extract (e.g., 1 for unigrams, 2 for bigrams)

      Returns:
          tuple: A tuple containing:
              - n_tokens (list): List of all n-gram tuples extracted from the text
              - vocab (set): Set of unique n-gram tuples
    """
    vocab = set()
    n_tokens = []
    tks = re.findall(r'\b[a-zA-Z]+\b', text.lower())
    for r in range(len(tks) - n + 1):
      n_gram = tuple(tks[r : r + n])
      n_tokens.append(n_gram)
      vocab.add(n_gram)
    return n_tokens, vocab

  vocab = set()
  tokens = []
  for txt in corpus:
    text = txt['text']
    # keep only bigram to represent features
    tks, voc = countFeaturesByNgram(text, 2)
    vocab.update(voc)
    tokens.append(tks)

  vocab = list(vocab)
  vocab_idx = {v: i for i, v in enumerate(vocab)}
  # create a sparse matrix
  texts = lil_matrix((len(corpus), len(vocab)), dtype=np.float64)
  for i in range(len(tokens)):
    tks = tokens[i]
    for tk in tks:
      texts[i, vocab_idx[tk]] += 1
  texts = texts.tocsr()
  return texts,genres,vocab


In [92]:
#given a numpy matrix representation of the features for the training set, the
# vector of true classes for each example, and the vocabulary as described
# above, this computes the accuracy of the model using leave one out cross
# validation and reports the most indicative features for each class
def evaluateModel(X,y,vocab,penalty="l1"):
  #create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=LeaveOneOut())

  #determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)

  #determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  neg_class_prob_sorted = model.coef_[0, :].argsort()
  pos_class_prob_sorted = (-model.coef_[0, :]).argsort()
  # pos_class_prob_sorted = (model.coef_[0, :]).argsort()

  termsToTake = 20
  pos_indicators = [vocab[i] for i in neg_class_prob_sorted[:termsToTake]]
  neg_indicators = [vocab[i] for i in pos_class_prob_sorted[:termsToTake]]

  return avg_score,pos_indicators,neg_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)


In [93]:
corpus = readShakespeare()

Run the following to train and evaluate two models with basic features:

In [94]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.653846
The most informative terms for pos are: ['helena', 'prospero', 'sir', 'duke', 'you', 'i', 'leontes', 'your', 'a', 'of', 'chastity', 'solemnities', 'imprison', 'farmhouse', 'honesty', 'dost', 'goeth', 'opposers', 'quarrelsome', 'laurence']
The most informative terms for neg are: ['him', 's', 'iago', 'our', 'imogen', 'brutus', 'lear', 'o', 'ham', 'rom', 'what', 'thy', 'and', 'the', 'produced', 'dost', 'goeth', 'opposers', 'quarrelsome', 'desk']
----------L2 Norm-----------
The model's average accuracy is 0.769231
The most informative terms for pos are: ['i', 'you', 'duke', 'prospero', 'a', 'helena', 'your', 'antonio', 'sir', 'leontes', 'hermia', 'for', 'lysander', 'ariel', 'sebastian', 'demetrius', 'camillo', 'stephano', 'me', 'parolles']
The most informative terms for neg are: ['iago', 'othello', 's', 'him', 'imogen', 'what', 'lear', 'brutus', 'his', 'cassio', 'o', 'ham', 'our', 'desdemona', 'rom', 'romeo', 'posthumus

Run the following to train and evaluate two models with features that are interesting for distinguishing comedy and tragedy:

In [95]:
X,y,vocab = createInterestingFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.730769
The most informative terms for pos are: [('not', 'a'), ('and', 'i'), ('you', 'have'), ('to', 'me'), ('the', 'duke'), ('she', 'is'), ('me', 'and'), ('if', 'you'), ('which', 'is'), ('i', 'am'), ('my', 'love'), ('the', 'king'), ('sir', 'toby'), ('of', 'syracuse'), ('as', 'you'), ('busines', 'i'), ('redemption', 'angelo'), ('ever', 'with'), ('accoutrements', 'as'), ('such', 'eyes')]
The most informative terms for neg are: [('the', 'gods'), ('art', 'thou'), ('the', 'moor'), ('give', 'me'), ('lady', 'macbeth'), ('to', 'the'), ('my', 'lord'), ('know', 'not'), ('orts', 'of'), ('dare', 'not'), ('worthy', 'fool'), ('me', 'madam'), ('courtesy', 'rom'), ('sleep', 'roderigo'), ('whom', 'three'), ('roman', 'yoke'), ('swords', 'bold'), ('busines', 'i'), ('impostor', 'that'), ('afeard', 'to')]
----------L2 Norm-----------
The model's average accuracy is 0.807692
The most informative terms for pos are: [('the', 'duke'), ('sir', 'toby

**TODO**: Based on the most informative features in the output of the classifier evaluation, what do these classifiers tell you about the differences between comedy and tragedy?

Based on most informative features from bigram features:
*   most features from unigram or bigram, if without given context, look neutral in sentiment.
*   Comedies: Feature characters like "the duke," "sir toby" (Twelfth Night), "princess of," "of france" (Love's Labour's Lost), "of syracuse" (Comedy of Errors) - showing lighter, romantic, or mistaken-identity plots.
*   Tragedies: Feature "my lord," "lady macbeth," "caesar's," "the moor" (Othello), "the gods" - showing more formal address, historical/political figures, and darker themes.
*   L1 Prefers Longer N-grams while L2 Prefers Shorter N-grams. This shows Shakespeare genre classification benefits from considering multiple bigram patterns together (L2 with bigrams).

| Feature Type | N-gram Range | L1 Accuracy | L2 Accuracy |
|:------------|:-------|------------:|------------:|
| Raw Frequencies | (1, 1) | 61.54% | 76.92% |
| Raw Frequencies | (2, 2) | 69.23% | **80.77%** |
| Raw Frequencies | (3, 3)| **76.92%** | 53.85% |